In [1]:
import os
os.remove('snorkel.db') #need these two files to exist in sandbox directory. Just copy from tutorials
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

In [3]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))


Corpus (Hardware) contains 100 Documents


In [4]:
from snorkel.models import candidate_subclass

Part = candidate_subclass('Part', ['part'])

from hardware_utils import OmniNgramsPart, OmniNgramsTemp

part_ngrams = OmniNgramsPart(n_max=3, split_tokens=None)

from hardware_utils import load_extended_parts_dict
from snorkel.matchers import DictionaryMatch, RegexMatchEach, RegexMatchSpan, Union

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
parts_dict = load_extended_parts_dict(filename)
print "Loaded %d part numbers." % len(parts_dict)

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,4}[\s]?[A-Z]{0,2}[0-9]?([-][A-Z0-9]{1,3})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,3})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

Loaded 1286 part numbers.


In [5]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, [part_ngrams], [parts_matcher])

In [6]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 2min 13s, sys: 964 ms, total: 2min 14s
Wall time: 2min 14s
Candidate Set (Hardware Candidates) contains 32201 Candidates


In [7]:
print len(candidates)
print candidates[0]

32201
Part(ImplicitSpan("MMBT3904B", parent=3589, words=[0,0], position=[5]))


In [8]:

# TEMP
from hardware_utils import entity_level_total_recall

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(candidates, filename, 'stg_temp_min', relation=False)

Preparing gold set...
Preparing candidates...
[========================================] 100%

Scoring on Entity-Level Total Recall
Entity-level Candidates extracted: 4633 
Entity-level Gold: 544
Intersection Candidates: 543
----------------------------------------
Overlap with Gold:  1.00

